In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import torch
import yaml
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

In [2]:
# Gestion du dossier de travail
if os.getcwd().endswith('notebooks'):
    os.chdir('..')
sys.path.append(os.getcwd())

In [3]:
from src.data_loaders.classification_ds import ArealData
from src.models.cnn_classifier import EuroSATCNN

In [4]:
with open("config.yaml", "r") as f:
        config = yaml.safe_load(f)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"--- Training on {device} ---")

# DataLoaders
train_dataset = ArealData(
    csv_file=config['classification']['data']['train_dir'],
    root_dir=config['classification']['data']['root_dir'],
    n_channels=config['classification']['model']['in_channels']
)

train_loader = DataLoader(
    train_dataset, 
    batch_size=config['classification']['model']['batch_size'], 
    shuffle=True,
    num_workers=2
)

validation_dataset = ArealData(
    csv_file=config['classification']['data']['validation_dir'],
    root_dir=config['classification']['data']['root_dir'],
    n_channels=config['classification']['model']['in_channels']
)

val_loader = DataLoader(
    validation_dataset, 
    batch_size=config['classification']['model']['batch_size'], 
    shuffle=True,
    num_workers=2
)

test_dataset = ArealData(
    csv_file=config['classification']['data']['test_dir'],
    root_dir=config['classification']['data']['root_dir'],
    n_channels=config['classification']['model']['in_channels']
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=config['classification']['model']['batch_size'], 
    shuffle=True,
    num_workers=2
)

--- Training on cpu ---


In [6]:
model = EuroSATCNN(
    in_channels=config['classification']['model']['in_channels'], 
    n_classes=config['classification']['model']['n_classes']
).to(device)
    
optimizer = torch.optim.Adam(model.parameters(), lr=config['classification']['model']['learning_rate'])
criterion = torch.nn.CrossEntropyLoss()


In [7]:
def evaluate( loader, desc="Evaluation"):
        model.eval()
        total_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in tqdm(loader, desc=desc, leave=False):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        return total_loss / len(loader), 100 * correct / total

In [8]:
model.train()
history = {'train_loss': [], 'val_acc': [], 'val_loss': []}
for epoch in range(config['classification']['model']['epochs']):
    running_loss = 0.0
    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{config['model']['epochs']}]", unit="batch")
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        current_loss = loss.item()
        running_loss += current_loss

        loop.set_postfix(loss=current_loss)

    avg_epoch_loss = running_loss / len(train_loader)
    # Validation
    val_loss, val_acc = evaluate(val_loader, desc="Validation")
    
    # Log History
    history['train_loss'].append(avg_epoch_loss)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    print(f"Epoch {epoch+1} Summary: Train Loss: {avg_epoch_loss:.4f} | Val Acc: {val_acc:.2f}%")


KeyboardInterrupt



In [ ]:
print("\n--- Final Test Evaluation ---")
loss, acc = evaluate(test_loader, desc="Test Set")
print(f"Final Results: Test Loss: {loss:.4f} | Test Acc: {acc:.2f}%")


--- Final Test Evaluation ---


Final Results: Test Loss: 0.6310 | Test Acc: 88.15%


In [ ]:
os.makedirs(os.path.dirname(config['classification']['model']['model_path']), exist_ok=True)
torch.save(model.state_dict(), config['classification']['model']['model_path'])
print("--- Training Complete & Model Saved ---")

--- Training Complete & Model Saved ---
